In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Neural Network Block 1
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0) # in- 28, out - 26, channels - 16, RF - 3 
        self.norm1 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout2d(p=0.15)

        # block 2
        self.conv2 = nn.Conv2d(16, 32, 3, padding=0) # 26, 24, 32, 5
        self.norm2 = nn.BatchNorm2d(32)
        self.dropout2 = nn.Dropout2d(p=0.15)

        # 1D convolution
        self.conv2_1d = nn.Conv2d(32, 10, 1) # 24, 24, 10, 

        # MAxpool
        self.pool1 = nn.MaxPool2d(2, 2) # 24, 12, 10, 
        
        # block 3
        self.conv3 = nn.Conv2d(10, 16, 3, padding=0) # 12, 10, 16, 
        self.norm3 = nn.BatchNorm2d(16)
        self.dropout3 = nn.Dropout2d(p=0.15)

        ## block 4 
        self.conv4 = nn.Conv2d(16, 32, 3, padding=0) #10, 8, 32, 
        self.norm4 = nn.BatchNorm2d(32)
        self.dropout4 = nn.Dropout2d(p=0.15)

        self.conv4_1d = nn.Conv2d(32, 10, 1) # 8, 8, 10

        # Max pooling
        self.pool2 = nn.MaxPool2d(2, 2) # 4, 4, 10
        
        self.conv7 = nn.Conv2d(32, 10, 3) #2, 2, 10

        # Global average pooling
        self.gap7 = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.dropout1(self.norm1(F.relu(self.conv1(x))))
        x = self.pool1(self.conv2_1d(self.dropout2(self.norm2(F.relu(self.conv2(x))))))
        x = self.dropout3(self.norm3(F.relu(self.conv3(x))))
        x = self.pool2((self.dropout4(self.norm4(F.relu(self.conv4(x))))))
        x = self.gap7(self.conv7(x))

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           4,640
       BatchNorm2d-5           [-1, 32, 24, 24]              64
         Dropout2d-6           [-1, 32, 24, 24]               0
            Conv2d-7           [-1, 10, 24, 24]             330
         MaxPool2d-8           [-1, 10, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           1,456
      BatchNorm2d-10           [-1, 16, 10, 10]              32
        Dropout2d-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 32, 8, 8]           4,640
      BatchNorm2d-13             [-1, 32, 8, 8]              64
        Dropout2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print(f'Current Epoch - {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

Current Epoch - 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0464, Accuracy: 9846/10000 (98.46%)

Current Epoch - 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0186, Accuracy: 9946/10000 (99.46%)

Current Epoch - 18


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0180, Accuracy: 9941/10000 (99.41%)

Current Epoch - 19


loss=0.0004614020581357181 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 55.74it/s]



Test set: Average loss: 0.0186, Accuracy: 9940/10000 (99.40%)

